# Predicting the Cause of Power Outages

**Name(s)**: Ethan Cao

**Website Link**: https://codingkomodo.github.io/power_out_model/

## Code

In [384]:
#Import Necessary Libraries
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### Framing the Problem

In [385]:
#Combing Date and Time as was done in our project 3

def combine_times(date_col_name, time_col_name, new_col_name, df):
    df = df.copy()
    df[new_col_name] = df[date_col_name] + pd.to_timedelta(df[time_col_name].astype(str))
    return df

data = pd.read_excel("outage.xlsx", skiprows=[0,1,2,3,4,6], index_col=1).iloc[:,1:]
data = combine_times("OUTAGE.START.DATE", 'OUTAGE.START.TIME', 'OUTAGE.START.DATETIME', data)
data = combine_times("OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME", "OUTAGE.RESTORATION.DATETIME", data)

In [386]:
data['CAUSE.CATEGORY'].unique()

array(['severe weather', 'intentional attack',
       'system operability disruption', 'equipment failure',
       'public appeal', 'fuel supply emergency', 'islanding'],
      dtype=object)

In [387]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaN,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


In [388]:
data['IS.HURRICANE'] = data['HURRICANE.NAMES'].isna() == False

In [389]:
data.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

In [390]:
data['IS.HURRICANE'].sum()

72

In [391]:
data['CAUSE.CATEGORY'].value_counts()

CAUSE.CATEGORY
severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: count, dtype: int64

In [392]:
793/1534

0.5169491525423728

In [393]:
data['ANOMALY.LEVEL'] = data['ANOMALY.LEVEL'].fillna(0)

In [394]:
px.bar(data,x='CAUSE.CATEGORY',y='CUSTOMERS.AFFECTED')

### Baseline Model

In [433]:
# Choose the features below
features = ['POSTAL.CODE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'MONTH', 'CAUSE.CATEGORY']

data_class = data[features].dropna()

X = data_class[['POSTAL.CODE', 'ANOMALY.LEVEL', 'MONTH']]
y = data_class['CAUSE.CATEGORY']

#split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#One Hot Encode Postal Code and pass the rest into a Random Forest Classifier
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE'])],
    remainder = 'passthrough')
pl = Pipeline([
    ('col_trans', col_trans),
    ('forest', RandomForestClassifier())
])

pl.fit(X_train, y_train)
prediction = pl.predict(X_test)


In [396]:
accuracy_score(y_test, prediction)

0.6655405405405406

In [397]:
import pprint as pp

In [398]:
# Create sin and cos graph for month to show circular data
sin_cos = (lambda x: pd.DataFrame({"sin_month":np.sin(2*np.pi*x['MONTH']/x['MONTH'].max()),"cos_month":np.cos(2*np.pi*x['MONTH']/x['MONTH'].max())}))(data)
fig = px.scatter(sin_cos,x="cos_month",y="sin_month",title="Cyclical Encoding of Months")

In [415]:
#fig.write_html('sig_graph.html', include_plotlyjs = 'cdn')

### Final Model

In [416]:
# Select the features below
features = ['POSTAL.CODE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'MONTH','CUSTOMERS.AFFECTED','OUTAGE.START.DATETIME', 'CAUSE.CATEGORY']
data_class = data[features].dropna()

X = data_class[['POSTAL.CODE', 'ANOMALY.LEVEL', 'MONTH','OUTAGE.DURATION','OUTAGE.START.DATETIME','CUSTOMERS.AFFECTED']]
y = data_class['CAUSE.CATEGORY']

#Split the data into test and training parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a column transformer that OHE Postal Code and splits month and hour into sin and cos columns which cyclical encode them
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE']),
    ('cyclical-encode-month', FunctionTransformer(lambda x: pd.DataFrame({"sin_month":np.sin(2*np.pi*x['MONTH']/x['MONTH'].max()),"cos_month":np.cos(2*np.pi*x['MONTH']/x['MONTH'].max())})), ['MONTH']),
    ('cyclical-encode-hour', make_pipeline(FunctionTransformer(lambda x:pd.DataFrame({"HOUR":x['OUTAGE.START.DATETIME'].apply(lambda y:y.hour)})),
                                            FunctionTransformer(lambda x: pd.DataFrame({"sin_hour":np.sin(2*np.pi*x['HOUR']/x['HOUR'].max()),"cos_hour":np.cos(2*np.pi*x['HOUR']/x['HOUR'].max())}))), ['OUTAGE.START.DATETIME'])],
    remainder = 'passthrough')

# Pass the columns into a RandomForestClassifier
pl = Pipeline([
    ('col_trans', col_trans),
    ('forest', RandomForestClassifier(max_depth = 20))
])

pl.fit(X_train, y_train)
prediction = pl.predict(X_test)


In [417]:
# Perform Grid Search to find the best cross validation amount and the best max depth for the Random Forest
param_grid = {"forest__max_depth": np.arange(5,30)}
search = GridSearchCV(pl, param_grid,cv=5)
search.fit(X_train,y_train)

c:\Users\ecao7\anaconda3\envs\dsc80\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning:

The least populated class in y has only 4 members, which is less than n_splits=5.



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one-hot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['POSTAL.CODE']),
                                                                        ('cyclical-encode-month',
                                                                         FunctionTransformer(func=<function <lambda> at 0x000001EB1ABB94C0>),
                                                                         ['MONTH']),
                                                                        ('cyclical-encode-hour',
                                                                         Pipeline(steps=[('functiontransformer-1',
                    

In [418]:
search.best_params_

{'forest__max_depth': 24}

In [419]:
search.cv_results_['mean_test_score']

array([0.81045365, 0.8258594 , 0.84124401, 0.84480135, 0.84953508,
       0.85664976, 0.85664976, 0.86256692, 0.86612426, 0.86730065,
       0.86730065, 0.86849112, 0.86966751, 0.86966751, 0.8672936 ,
       0.86848408, 0.87204846, 0.86967456, 0.86612426, 0.87322485,
       0.86374331, 0.86966751, 0.86849817, 0.87085094, 0.87085799])

In [420]:
accuracy_score(y_test, search.predict(X_test))

0.8773584905660378

In [421]:
search.score(X_train, y_train)

1.0

### Fairness Analysis

In [422]:
# Check if each prediction aligns with the actual value
check_diff_class = X_test.assign(prediction = prediction).assign(actual = y_test)
check_diff_class['Predict_right'] = check_diff_class['prediction'] == check_diff_class['actual']
check_diff_class

,POSTAL.CODE,ANOMALY.LEVEL,MONTH,OUTAGE.DURATION,OUTAGE.START.DATETIME,CUSTOMERS.AFFECTED,prediction,actual,Predict_right
OBS,,,,,,,,,
993,LA,0.3,6.0,8054.0,2004-06-02 01:46:00,59057.0,severe weather,severe weather,True
560,MD,-0.1,6.0,0.0,2012-06-06 12:37:00,0.0,intentional attack,intentional attack,True
1395,CT,0.4,6.0,4560.0,2009-06-26 17:00:00,50752.0,severe weather,severe weather,True
556,MD,-0.5,2.0,1.0,2014-02-05 08:05:00,181000.0,severe weather,severe weather,True
131,MI,-0.4,7.0,5580.0,2008-07-02 15:00:00,239663.0,severe weather,severe weather,True
...,...,...,...,...,...,...,...,...,...
613,PA,-0.6,2.0,5160.0,2009-02-12 08:00:00,132000.0,severe weather,severe weather,True
903,DE,1.0,6.0,4254.0,2015-06-23 17:06:00,65000.0,severe weather,severe weather,True
1211,CA,1.2,7.0,44.0,2015-07-27 03:52:00,484.0,islanding,islanding,True


In [423]:
check_diff_class['actual'].unique()

array(['severe weather', 'intentional attack',
       'system operability disruption', 'equipment failure',
       'public appeal', 'islanding', 'fuel supply emergency'],
      dtype=object)

In [424]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME,IS.HURRICANE
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,False
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,False
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,False
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,False
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00,False
1531,2006,NaN,North Dakota,ND,MRO,West North Central,0.0,NaN,NaT,NaN,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT,False
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00,False


In [425]:
# Imports for a confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
#Find the accuracy by group
group_accuracies = check_diff_class[['prediction', 'actual', 'Predict_right']].groupby('actual')['Predict_right'].mean().sort_values(ascending=False)

# Plotting the accuracy for each group as a bar plot
fig = px.bar(x=group_accuracies.index, y=group_accuracies.values,
             labels={'x': 'Postal Code', 'y': 'Accuracy'},
             title='Accuracy by Postal Code Groups')

fig.update_traces(marker_color='rgb(0,0,140)')  # Change bar color if desired
fig.update_xaxes(tickangle=45)
fig.show()

In [426]:
#fig.write_html('accuracy_bar.html', include_plotlyjs='cdn')

In [427]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Replace 'y_test' and 'prediction' with your actual test labels and predictions
# Example data (replace this with your actual data)


# Replace these labels with your specific category names
category_labels = [
    'equipment failure',
    'fuel supply emergency',
    'intentional attack',
    'islanding',
    'public appeal',
    'severe weather',
    'system operability disruption'
]


# Calculate the confusion matrix
cm = confusion_matrix(check_diff_class['actual'], check_diff_class['prediction'])

# Create a Plotly heatmap for the confusion matrix
fig = ff.create_annotated_heatmap(z=cm[::-1,:], x=category_labels, y=category_labels[::-1], colorscale='Blues')

# Update the layout
fig.update_layout(title='Confusion Matrix',
                  xaxis=dict(title='Predicted Label'),
                  yaxis=dict(title='True Label'))

# Display the confusion matrix
fig.show()
#fig.write_html('confusion_matrix.html', include_plotlyjs='cdn')


In [428]:
cm[::-1,::-1]

array([[  9,   9,   0,   0,   1,   0,   0],
       [  0, 133,   0,   0,   0,   0,   0],
       [  0,   4,   1,   0,   0,   0,   0],
       [  0,   1,   0,   6,   0,   0,   0],
       [  0,   1,   0,   0,  40,   0,   0],
       [  0,   0,   0,   0,   1,   0,   0],
       [  1,   5,   0,   0,   0,   0,   0]], dtype=int64)

In [429]:
from sklearn.metrics import precision_score, recall_score

# Assuming 'y_test' contains the true labels and 'prediction' contains predicted labels
precision = precision_score(y_test, prediction, average='weighted')
recall = recall_score(y_test, prediction, average='weighted')

c:\Users\ecao7\anaconda3\envs\dsc80\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [430]:
precision

0.866801877983898

In [431]:
recall

0.8915094339622641

In [432]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# Sample data (replace this with your actual dataset)
df = check_diff_class

# Binarize 'prediction' into 'severe_weather' (1 for 'severe weather', 0 otherwise)
df['severe_weather'] = np.where(df['prediction'] == 'severe weather', 1, 0)

# Calculate the observed accuracy difference between 'severe weather' and 'not severe weather'
observed_accuracy_diff = df[df['severe_weather'] == 1]['severe_weather'].mean() - \
                         df[df['severe_weather'] == 0]['severe_weather'].mean()

# Permutation test to check if there's a significant difference
num_permutations = 1000
accuracy_diffs = []

for _ in range(num_permutations):
    # Shuffle the 'severe_weather' labels
    shuffled_labels = np.random.permutation(df['severe_weather'].values)
    
    # Assign shuffled labels back to the DataFrame
    shuffled_df = df.assign(severe_weather=shuffled_labels)
    
    # Calculate accuracy difference for shuffled labels
    accuracy_diff = shuffled_df[shuffled_df['severe_weather'] == 1]['severe_weather'].mean() - \
                    shuffled_df[shuffled_df['severe_weather'] == 0]['severe_weather'].mean()
    
    accuracy_diffs.append(accuracy_diff)

# Calculate p-value
p_value = (np.abs(accuracy_diffs) >= np.abs(observed_accuracy_diff)).mean()

print(f"Observed accuracy difference: {observed_accuracy_diff:.4f}")
print(f"P-value: {p_value:.4f}")


Observed accuracy difference: 1.0000
P-value: 1.0000
